In [ ]:
import numpy as np
import os
import json

from func.nets import vid_enc, vid_enc_vgg19
from chainer import serializers

import sys
sys.path.append('script/')
from summarize import get_flabel
from func.sampling.vsum import VSUM

import chainer
from chainer import configuration

In [2]:
# settings
feat_type = 'smt_feat' # smt_feat (proposed) or vgg

# load embedding model
if feat_type == 'smt_feat':
    model = vid_enc.Model()
    serializers.load_npz('data/trained_model/model_par', model)
elif feat_type == 'vgg':
    model = vid_enc_vgg19.Model()
else:
    raise RuntimeError('[invalid feat_type] use smt_feat or vgg')

In [3]:
# prepair output dir
d_name = 'summe'
dataset_root = 'data/{}/'.format(d_name)
out_dir = 'results/{:}/{:}/'.format(d_name, feat_type)
print 'save to: ', out_dir

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

save to:  results/summe/smt_feat/


In [4]:
# load dataset metadata
dataset = json.load(open(dataset_root + 'dataset.json'))
video_id = [d['videoID'] for d in dataset]

print 'Video list:'
for vi in video_id:
    print '-', vi

Video list:
- Kids_playing_in_leaves
- playing_ball
- Paintball
- Cooking
- Scuba
- Cockpit_Landing
- Fire Domino
- Air_Force_One
- Bus_in_Rock_Tunnel
- Base jumping
- Jumps
- Saving dolphines
- Bearpark_climbing
- Uncut_Evening_Flight
- Valparaiso_Downhill
- Eiffel Tower
- Excavators river crossing
- St Maarten Landing
- Playing_on_water_slide
- Statue of Liberty
- Car_railcrossing
- paluma_jump
- Notre_Dame
- car_over_camera
- Bike Polo


In [5]:
# summarize video
v_id = 'Cooking'

with configuration.using_config('train', False):
    with chainer.no_backprop_mode():
        vsum = VSUM(v_id, model, dataset=d_name, seg_l=5)

_, frames, _ = vsum.summarizeRep(seg_l=5, weights=[1.0, 0.0])

# get 0/1 label for each frame
fps = vsum.dataset.data['fps']
fnum = vsum.dataset.data['fnum']
label = get_flabel(frames, fnum, fps, seg_l=5)

load Cooking
budget:  2


/home/mayu-ot/miniconda2/envs/accv_vsum/lib/python2.7/site-packages/numpy/core/_methods.py:26: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return umr_maximum(a, axis, None, out, keepdims)


In [6]:
# write summarized video
from skvideo.io import vread, vwrite

video_path = '/home/mayu-ot/Data/Dataset/summe/video/%s.webm' % v_id

video_data = vread(video_path)
sum_vid = video_data[label.ravel().astype(np.bool), :,:,:]

print 'writing video to', 'sum_%s.mp4'%v_id
vwrite('sum_%s.mp4'%v_id, sum_vid)

writing video to sum_Cooking.mp4
